In [ ]:
pip install datasets evaluate transformers rouge-score nltk

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.3 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=e39f416e20209eb44f2cd1a9d909332050407391ff9d7d8ab04231a7e5a7f64e
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's depend

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Then you need to install Git-LFS. Uncomment the following instructions:

In [ ]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


Make sure your version of Transformers is at least 4.11.0 since the functionality was introduced in that version:

In [ ]:
import transformers

print(transformers.__version__)

4.44.2


In [ ]:
model_checkpoint = "t5-small"

This notebook is built to run  with any model checkpoint from the [Model Hub](https://huggingface.co/models) as long as that model has a sequence-to-sequence version in the Transformers library. Here we picked the [`t5-small`](https://huggingface.co/t5-small) checkpoint.

## Loading the dataset

In [ ]:
import pandas as pd
from evaluate import load
data = pd.read_excel('/content/PROJECT_DATA.xlsx')

# Load the ROUGE metric
metric = load("rouge")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
data.head(2)

,SNO.,MCV,MCHC,HB,RBC,WBC,PLT,RDWCV,NEUTRO,LYMPHO,SUMMARY
0,Report1,87.7,30.1,7.3,2.77,10.0,189.0,11.4,50.1,43.2,Your report shows a hemoglobin level of 7.3 g/...
1,Report2,88.2,20.2,7.3,2.84,10.0,180.0,11.4,52.3,42.4,"In your CBC report, your hemoglobin level is 7..."


In [ ]:
# Check column names
print(list(data.columns))


['SNO.', 'MCV ', 'MCHC ', 'HB ', 'RBC ', 'WBC ', 'PLT', 'RDWCV', 'NEUTRO', 'LYMPHO', 'SUMMARY ']


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [ ]:
if model_checkpoint in ["t5-small", "t5-base", "t5-larg", "t5-3b", "t5-11b"]:
    prefix = "summarize: "
else:
    prefix = ""

In [ ]:
from datasets import Dataset
dataset = Dataset.from_pandas(data)


In [ ]:
def preprocess_function(examples):
    # Concatenate the features into a single string for the input
    input_texts = [
        " ".join([
            str(examples['MCV '][i]),
            str(examples['MCHC '][i]),
            str(examples['HB '][i]),
            str(examples['RBC '][i]),
            str(examples['WBC '][i]),
            str(examples['PLT'][i]),
            str(examples['RDWCV'][i]),
            str(examples['NEUTRO'][i]),
            str(examples['LYMPHO'][i])
        ])
        for i in range(len(examples['MCV ']))  # Iterate over the batch
    ]

    # Tokenize the inputs
    model_inputs = tokenizer(input_texts, max_length=1024, padding="max_length", truncation=True)

    # Tokenize the targets (adjust to the correct column name)
    labels = tokenizer(examples['SUMMARY '], max_length=200, padding="max_length", truncation=True)

    # Add labels to the model inputs
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
print(data.columns)

Index(['SNO.', 'MCV ', 'MCHC ', 'HB ', 'RBC ', 'WBC ', 'PLT', 'RDWCV',
       'NEUTRO', 'LYMPHO', 'SUMMARY '],
      dtype='object')


In [ ]:
# Apply preprocessing
tokenized_datasets = dataset.map(preprocess_function, batched=True)

# Train/Test Split using Hugging Face's `train_test_split`
train_test_split_datasets = tokenized_datasets.train_test_split(test_size=0.2)

# Extract train and test datasets
train_dataset = train_test_split_datasets['train']
test_dataset = train_test_split_datasets['test']


Map:   0%|          | 0/282 [00:00<?, ? examples/s]

In [ ]:
from transformers import T5ForConditionalGeneration

model_checkpoint = "t5-small"  # Or other T5 models you prefer
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint)


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
!pip install evaluate
import nltk
import numpy as np
import evaluate

# Load ROUGE metric
metric = evaluate.load("rouge")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in labels as they cannot be decoded
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Compute ROUGE score
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)

    result = {key: value * 100 for key, value in result.items()}

    # Compute the average generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}


In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import DataCollatorForSeq2Seq
import nltk
import pickle

nltk.download('punkt')
nltk.download('all')

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",  # Save model here
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    predict_with_generate=True,
    fp16=True,
    save_total_limit=3
)

# Data collator for Seq2Seq tasks
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Define the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics  # Ensure this is adjusted for your task
)

# Start training
trainer.train()

# Save the trained model and tokenizer
model.save_pretrained("./results/trained_model")  # Save model in HuggingFace format
tokenizer.save_pretrained("./results/trained_model")  # Save tokenizer

# Optional: Save model as a .pkl file (this is not standard for Hugging Face models)
with open("./results/trained_model/model.pkl", "wb") as f:
    pickle.dump(model, f)




[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downlo

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,4.672013,3.261100,1.081100,2.919700,3.301400,4.877200
2,No log,3.238100,2.953000,1.047300,2.548800,2.968100,3.578900
3,No log,2.770674,2.257500,0.838300,1.966000,2.365600,3.000000
4,5.153500,2.391119,0.755300,0.329900,0.613100,0.755300,0.666700
5,5.153500,2.157582,0.000000,0.000000,0.000000,0.000000,0.000000
6,5.153500,1.965529,0.000000,0.000000,0.000000,0.000000,0.000000
7,2.634700,1.785369,0.000000,0.000000,0.000000,0.000000,0.000000
8,2.634700,1.635868,0.048400,0.000000,0.048400,0.048400,0.666700
9,2.634700,1.525814,0.000000,0.000000,0.000000,0.000000,1.000000
10,2.634700,1.439612,0.525900,0.154200,0.359800,0.402600,3.000000


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control

In [ ]:
# Save the model locally after training
model.save_pretrained('C:\\Users\\sunil\\Desktop\\MODEL3')

# Save the tokenizer (if needed)
tokenizer.save_pretrained("C:\\Users\\sunil\\Desktop\\MODEL3")


('C:\\Users\\sunil\\Desktop\\MODEL3/tokenizer_config.json',
 'C:\\Users\\sunil\\Desktop\\MODEL3/special_tokens_map.json',
 'C:\\Users\\sunil\\Desktop\\MODEL3/spiece.model',
 'C:\\Users\\sunil\\Desktop\\MODEL3/added_tokens.json',
 'C:\\Users\\sunil\\Desktop\\MODEL3/tokenizer.json')

In [ ]:
import torch
import pickle

# Define your save path
save_path = "C:\\Users\\sunil\\Desktop\\MODEL3"

# Save the Hugging Face model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Save the model as a .pkl file
pkl_save_path = f"{save_path}\\model.pkl"
with open(pkl_save_path, "wb") as f:
    pickle.dump(model, f)

# Verify if the model is saved correctly
print(f"Model saved in Hugging Face format at: {save_path}")
print(f"Model also saved as .pkl file at: {pkl_save_path}")


Model saved in Hugging Face format at: C:\Users\sunil\Desktop\MODEL3
Model also saved as .pkl file at: C:\Users\sunil\Desktop\MODEL3\model.pkl


In [ ]:
import torch
import pickle

# Define your save path
save_path = "C:\\Users\\sunil\\Desktop\\MODEL3"

# Save the Hugging Face model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Save the model's state_dict as a .pt file
pt_save_path = f"{save_path}\\model.pt"
torch.save(model.state_dict(), pt_save_path)

# Optionally, you can also save the tokenizer separately as a pickle file
pkl_save_path = f"{save_path}\\model.pkl"
with open(pkl_save_path, "wb") as f:
    pickle.dump(model, f)

# Verify if the model is saved correctly
print(f"Model saved in Hugging Face format at: {save_path}")
print(f"Model saved as .pt file at: {pt_save_path}")
print(f"Model also saved as .pkl file at: {pkl_save_path}")


Model saved in Hugging Face format at: C:\Users\sunil\Desktop\MODEL3
Model saved as .pt file at: C:\Users\sunil\Desktop\MODEL3\model.pt
Model also saved as .pkl file at: C:\Users\sunil\Desktop\MODEL3\model.pkl


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load trained model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained('C:\\Users\\sunil\\Desktop\\MODEL3')
tokenizer = AutoTokenizer.from_pretrained('C:\\Users\\sunil\\Desktop\\MODEL3')

# Sample test data
test_data = ["86.5	30.1	1.8	4.15	10.9	158	12.8	71.9	18.1	"]

# Tokenize test data
inputs = tokenizer(test_data, return_tensors="pt", padding=True, truncation=True)

# Use the correct 'input_ids' from the tokenized data
generated_output = model.generate(
    input_ids=inputs['input_ids'],  # Corrected to use inputs['input_ids']
    max_length=150,  # Set maximum length for the generation
    max_new_tokens=400,  # This limits only the number of new tokens generated
    num_beams=4,  # Optional: Set beam search for higher quality output
    early_stopping=True  # Optional: Stop generation if the model thinks it's done
)

# Decode the generated output
generated_text = tokenizer.decode(generated_output[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)


Both `max_new_tokens` (=400) and `max_length`(=150) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


The report shows a hemoglobin level of 1.8 g/dL and an RBC count of 4.15 million cells/L, which is normal, with neutrophils at 71.9% and lymphocytes at 11%. The platelet count of 10.9 thousand cells/L is normal, with neutrophils at 71.9% and lymphocytes at 11%. The platelet count of 158 thousand cells/L is normal, with neutrophils at 71.9% and lymphocytes at 10.9 reflects a balanced immune system. Overall, this profile suggests mild anemia.
